In [ ]:
import picos as pic
import cvxopt as cvx

import numpy as np
from numpy import linalg as LA
from scipy.special import comb as nchoosek # n Choose k (n ueber k)

In [ ]:
### Seite 37 (im Text zwischen (4.12) und (4.13))
def _N(n):
    return np.diag([p for p in xrange(-n, 0, 1)])

### Seite 55; (4.64)
def _M(n):
    return np.diag([p for p in xrange(0, n, 1)])

### Seite 55; (4.64)
def _P(l, k, n):
    I = np.eye(n)
    Mn = _M(n)
    P = I
    if k == 0:
        pass # P=I
    else:
        for q in xrange(0, k, 1):
            P = P * ((l-q)*I + Mn)
    return cvx.matrix(P)

In [ ]:
##############################
# Boris Paper UBoot          #
##############################
A = np.matrix([[0, 1, 0],
              [0, 0, 1],
              [0, 0, -0.005]])
a = -A[-1][:].T ### !!!!!
b = np.matrix([[0],[0],[1]])
c = np.matrix([1, 0, 0])
u_max = 2.5e-5
n = 3

X0 = [np.matrix([-10, -0.05, -0.0046]).T,
      np.matrix([-10, -0.05, 0.0046]).T,
      np.matrix([-10, 0.05, -0.0046]).T,
      np.matrix([10, -0.05, 0.0046]).T,
      np.matrix([10, -0.05, -0.0046]).T,
      np.matrix([10, 0.05, 0.0046]).T]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c

N = cvx.matrix(_N(n))
M = cvx.matrix(_M(n))

##### Entwurf parameter #####
beta = 2 # beta >=1 !

In [ ]:
# S. 78 Boris (LMI-Entwurf)
def convex_problem(gamma, mu=1):
    ##############################
    # Convex Problem (35)        #
    ##############################
    prob = pic.Problem()

    # Constants
    AA = pic.new_param('A', A)
    II = pic.new_param('I_n', np.eye(n))
    III = pic.new_param('I_n+1', np.eye(n+1))
    aa = pic.new_param('a', a)
    bb = pic.new_param('b', b)
    XX0 = pic.new_param('X0', X0)

    NN = pic.new_param('N', N)
    MM = pic.new_param('M', M)

    AA0 = pic.new_param('A0', AA+bb*aa.T)

    ## REMARK THIS!!!! gamma is optimization variable but not convex, thus to be bisected to find "bigg-ish" value
    gamma = pic.new_param('gamma', gamma)

    # Problem
    prob = pic.Problem()

    # Parameters
    QQ = prob.add_variable('Q', (n, n), vtype='symmetric')
    zz = prob.add_variable('z', n)
    zz_star = prob.add_variable('z_star', n)

    # Objective
    prob.set_objective('find', None)

    # Constraints
    #prob.add_constraint(QQ >> 0)
    #prob.add_constraint(QQ*NN + NN*QQ << 0)
    prob.add_constraint(QQ*AA0.T + AA0*QQ - zz*bb.T - bb*zz.T << 0)

    ## (31) ???

    prob.add_list_of_constraints([((1          & XX0[i].T) //
                                   (XX0[i]     & QQ      )) >> 0
                                       for i in range(0, len(X0))])

    prob.add_constraint(QQ*AA0.T + AA0*QQ - zz_star*bb.T - bb*zz_star.T << -2*(gamma*QQ))
    prob.solve(verbose=0, solver='cvxopt')
    return prob

In [ ]:
## Lets bisect
# Expects min_val to be valid, and max_val to be not valid
# func only taking on (scalar) argument -> gamma
def bisect_prob(min_val, max_val, func, diff=1e-5, max_iter=50, _iteration=0):
    if _iteration > max_iter: 
        print "Iter:", _iteration
        print "Diff:", (max_val - min_val)/2.0
        return min_val, prob
    elif (max_val - min_val)/2.0 <= diff:
        print "Iter:", _iteration
        print "Diff:", (max_val - min_val)/2.0
        return min_val, prob
    else:
        mid_val = min_val+(max_val - min_val)/2.0
        #print "1. Evaluating: ", mid_val
        try:
            prob = func(mid_val)
            Q = prob.get_valued_variable('Q')
            z = prob.get_valued_variable('z')
            z_star = prob.get_valued_variable('z_star')
        except Exception as e:
            #print "Problem not solved!"
            #print e
            max_val = mid_val
        else:
            #print "Problem solved!"
            #print Q, z, z_star
            min_val = mid_val
        finally:
            #print "2. Recursing: ", min_val, max_val
            return bisect_prob(min_val, max_val, func, diff, max_iter,_iteration+1)
        #return min_val, max_val
    

In [ ]:
#%%timeit ~4sec
val, prob = bisect_prob(1, 50, convex_problem)
print val
print convex_problem(val).get_valued_variable('Q')

In [ ]:
prob = convex_problem(5.1)
Q = None
z = None
z_star = None
try:
    Q = prob.get_valued_variable('Q')
    z = prob.get_valued_variable('z')
    z_star = prob.get_valued_variable('z_star')
except Exception as e:
    print "Problem not solved!"
    print e
else:
    print "Problem solved!"
    print Q, z, z_star
#print prob.get_valued_variable('z')
#print prob.get_valued_variable('z_star')

#print "P:\n", PP
#print "eig:\n", LA.eigvals(PP.value)

#print 
#print "AP+PA:\n",(AA.T*PP + PP*AA).value
#print "eig:\n", LA.eigvals((AA.T*PP + PP*AA).value)

In [ ]:
## Seite 47 Boris
# Wähle p_min, mu_start, zeta, zeta_start
def step1(p_min, mu_start, zeta, zeta_start):
    # Löse (A.15) mit mu = 1 -> l_star -> lambda_hat_i(1)=lambda_i(A-b*l_star.T)
    